<a href="https://colab.research.google.com/github/Orangefox76200/EMD-CapsNet-LSTM/blob/main/EMD_CapsNet_LSTM_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import keras

In [3]:
from keras.models import Sequential

In [4]:
from keras.layers import Conv1D

In [5]:
from keras.layers import LSTM

In [6]:
from keras.layers import Dense